# 🧙‍♂️ Sample Generation with Pretrained Model + LLLA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Jac-Zac/PML_DL_Final_Project/blob/master/notebooks/notebook_llla_flow.ipynb)

In [ ]:
# NOTE: Initial import needed for Google Collab
# !pip install laplace-torch

### Initial setup ⚙️

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    import os
    !pip install laplace-torch -q 
    repo_dir = "PML_DL_Final_Project"
    
    if not os.path.exists(repo_dir):
        !git clone https://github.com/Jac-Zac/PML_DL_Final_Project.git
    else:
        print(f"Repository '{repo_dir}' already exists. Skipping clone.")

        
    if os.path.isdir(repo_dir):
        %cd $repo_dir
        !pip install dotenv -q
    else:
        print(f"Directory '{repo_dir}' not found. Please clone the repository first.")

else:
    %cd ..

### 📦 Imports

In [ ]:
import torch
import os
from PIL import Image
from src.models.flow import UQFlowMatching
from src.utils.data import get_llla_dataloader
from src.utils.plots import plot_interleaved_image_uncertainty, plot_uncertainty_sums
from src.utils.environment import get_device, set_seed, load_pretrained_model

### 🧪 Setup: Seed and Device

In [ ]:
seed = 1337
set_seed(seed)
device = get_device()
os.makedirs("checkpoints", exist_ok=True)

## 💡 Image Generation

#### 🛠️ Configuration Parameters

In [ ]:
sample_batch_size = 16
num_classes = 10
time_emb_dim = 128
save_dir = "samples"
model_name = "unet"
method = "flow"
ckpt_path = "jac-zac/bayesflow-project/best-model:v19"  # or use your last checkpoint
# ckpt_path = "jac-zac/diffusion-project/best-model:v83"  # Working mnist model 
use_wandb = True
dataset_name = 'FashionMNIST'

#### 📥 Load model and Dataloader

In [ ]:
from src.models.llla_model import LaplaceApproxModel
from types import SimpleNamespace

model_kwargs={
    "num_classes": num_classes, "time_emb_dim": time_emb_dim,
    # NOTE: We are currently using different time embedding because of a small bug but it is fine
    "time_embedding_type": "mlp" if method == "flow" else "sinusoidal"
}

# Load pretrained MAP model using best checkpoint
flow_model = load_pretrained_model(
    model_name=model_name,
    ckpt_path=ckpt_path,
    device=device,
    model_kwargs=model_kwargs,
    use_wandb=use_wandb,
)

# Prepare data loaders for the Laplace fit
train_loader, _ = get_llla_dataloader(batch_size=sample_batch_size, dataset_name = dataset_name, mode = "flow")

mnist_config = SimpleNamespace()
mnist_config.data = SimpleNamespace()
mnist_config.data.image_size = 28  # MNIST image size

### 💪 Fit Laplace approximation

In [ ]:
# Wrap diffusion model with your Custom Model for Laplace last layer approx
# NOTE: Automatically call fit
laplace_model = LaplaceApproxModel(
    flow_model, train_loader, args=None, config=mnist_config
)

print("Laplace fitting completed on last layer of the diffusion model.")

### 💨 Initialize Flow Process

In [ ]:
total_steps = 50
n_samples = 2
cov_num_sample = 100

# Initialize uncertainty-aware diffusion (same interface as base class)
flow = UQFlowMatching(img_size=mnist_config.data.image_size, device=device)

# Generate samples and uncertainty for images for different classes
all_samples_grouped, uncertainties = flow.sample_with_uncertainty(
    model=laplace_model,
    y=torch.arange(n_samples, device=device) % num_classes,
    cov_num_sample=cov_num_sample,
    num_steps=total_steps,
    log_intermediate=True,
)

In [ ]:
interval = 4

# Now call the interleaved plot function
plot_interleaved_image_uncertainty(
    images=all_samples_grouped,       # (T, B, C, H, W)
    uncertainties=uncertainties,      # (T, B, C, H, W)
    save_path=os.path.join(save_dir, "interleaved_grid.png"),
    timesteps=list(range(0, total_steps, interval)),
    uq_cmp="viridis",                 # Or "grey" or other matplotlib colormap
)

In [ ]:
# Display samples grid images and uncertainty
out_path_img = os.path.join(save_dir, "interleaved_grid.png")
display(Image.open(out_path_img))

In [ ]:
# plot_uncertainty_sums(uncertainties, samples=[0,2,5]), save_path="uncertainty_plot.png")  # save to file only specific samples
plot_uncertainty_sums(uncertainties)  # all samples